In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
for name,_ in model.named_children():
    print(name)

model.children

conv1
pool
conv2
fc1
fc2
fc3


<bound method Module.children of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [4]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)

model.pool.register_forward_hook(layer1_hook)

In [5]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [6]:
outputs

[tensor([[[[1.6354, 0.0000, 0.3015,  ..., 0.3398, 0.0000, 0.6202],
           [0.6096, 1.0233, 0.2651,  ..., 0.2344, 0.2728, 0.0027],
           [1.0590, 0.9741, 0.2907,  ..., 0.7530, 0.2738, 0.4453],
           ...,
           [0.0000, 0.4490, 1.4842,  ..., 0.8040, 0.3499, 0.9660],
           [0.7111, 0.5603, 0.0892,  ..., 0.0238, 0.0000, 0.2151],
           [0.7337, 1.0385, 0.0322,  ..., 1.1635, 0.1962, 0.8472]],
 
          [[0.1421, 0.6391, 0.6246,  ..., 0.2168, 0.1412, 0.3752],
           [0.7152, 0.1153, 0.8292,  ..., 0.9035, 0.9681, 0.4326],
           [0.7761, 0.4431, 0.7572,  ..., 0.2474, 0.0288, 0.3833],
           ...,
           [0.0679, 0.1868, 0.8185,  ..., 0.5477, 0.7323, 0.3987],
           [0.6865, 0.0391, 0.3622,  ..., 0.0000, 0.9076, 1.1645],
           [0.9711, 0.5825, 0.8095,  ..., 0.5533, 0.2704, 0.4104]],
 
          [[0.1928, 0.6503, 0.1903,  ..., 0.3488, 0.3337, 0.4096],
           [0.7885, 0.0419, 0.5395,  ..., 1.0474, 1.0857, 1.0585],
           [0.3257, 0.07

In [11]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [14]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [15]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


